In [1]:
import pandas as pd
import geopandas as gpd
from siuba import *
import shared_utils

from calitp_data_analysis import get_fs
from segment_speed_utils import helpers, time_series_utils, gtfs_schedule_wrangling
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT, analysis_date

In [2]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [3]:
catalog.stop_segments

{'dir': '${gcs_paths.SEGMENT_GCS}', 'stage1': '${speeds_tables.vp_dwell}', 'stage2': 'nearest/nearest_vp_shape_segments', 'stage2b': 'nearest/nearest2_vp_shape_segments', 'stage3': 'stop_arrivals', 'stage4': 'speeds_stop_segments', 'trip_stop_cols': ['trip_instance_key', 'stop_sequence'], 'shape_stop_cols': ['shape_array_key', 'shape_id', 'stop_sequence'], 'stop_pair_cols': ['stop_pair', 'stop_pair_name'], 'route_dir_cols': ['route_id', 'direction_id'], 'shape_stop_single_segment': 'rollup_singleday/speeds_shape_stop_segments', 'route_dir_single_segment': 'rollup_singleday/speeds_route_dir_segments', 'route_dir_multi_segment': 'rollup_multiday/speeds_route_dir_segments', 'segments_file': 'segment_options/shape_stop_segments', 'max_speed': '${speed_vars.max_speed}'}

In [4]:
analysis_date

'2024-10-16'

## wait, isn't 'rt_stop_times' what I need?

* I basically need the same data Amanda's using for bunching, except I do also care about schedule

In [5]:
catalog.rt_stop_times

{'dir': '${gcs_paths.SEGMENT_GCS}', 'stage1': '${speeds_tables.vp_dwell}', 'stage2': 'nearest/nearest_vp_rt_stop_times', 'stage2b': 'nearest/nearest2_vp_rt_stop_times', 'stage3': 'rt_stop_times/stop_arrivals', 'stage4': 'rt_stop_times/speeds', 'trip_stop_cols': ['trip_instance_key', 'stop_sequence'], 'shape_stop_cols': ['shape_array_key', 'shape_id', 'stop_sequence'], 'stop_pair_cols': ['stop_pair', 'stop_pair_name'], 'route_dir_cols': ['route_id', 'direction_id'], 'segments_file': 'segment_options/stop_segments', 'trip_speeds_single_summary': 'rollup_singleday/speeds_trip', 'route_dir_single_summary': 'rollup_singleday/speeds_route_dir', 'route_dir_multi_summary': 'rollup_multiday/speeds_route_dir', 'min_trip_minutes': '${speed_vars.time_min_cutoff}', 'max_trip_minutes': 180, 'max_speed': '${speed_vars.max_speed}', 'min_meters_elapsed': 1609}

In [6]:
catalog.rt_vs_schedule_tables

{'dir': '${gcs_paths.RT_SCHED_GCS}', 'stop_times_direction': 'stop_times_direction', 'sched_trip_metrics': 'schedule_trip/schedule_trip_metrics', 'sched_route_direction_metrics': 'schedule_route_dir/schedule_route_direction_metrics', 'vp_trip_metrics': 'vp_trip/trip_metrics', 'vp_route_direction_metrics': 'vp_route_dir/route_direction_metrics', 'vp_operator_metrics': 'vp_operator/operator_metrics', 'sched_stop_metrics': 'schedule_stop/schedule_stop_metrics', 'schedule_rt_stop_times': 'schedule_rt_stop_times', 'early_trip_minutes': -5, 'late_trip_minutes': 5}

In [7]:
path = f'{catalog.schedule_downloads.dir}{catalog.schedule_downloads.trips}_{analysis_date}.parquet'

In [8]:
path

'gs://calitp-analytics-data/data-analyses/rt_delay/compiled_cached_views/trips_2024-10-16.parquet'

In [9]:
trips = pd.read_parquet(path)

In [12]:
path = f'{catalog.rt_vs_schedule_tables.dir}{catalog.rt_vs_schedule_tables.schedule_rt_stop_times}_{analysis_date}.parquet'

In [14]:
schedule_rt = pd.read_parquet(path)

In [15]:
schedule_rt >> head(3)

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec
0,t_5912118_b_83127_tn_0,7728100,5,25426.0,09e16227fc42c4fe90204a9d11581034,893cb960f2ca78f823bd67adee30eaa3,25434
1,t_5912118_b_83127_tn_0,7728400,8,25559.0,09e16227fc42c4fe90204a9d11581034,893cb960f2ca78f823bd67adee30eaa3,25594
2,t_5912118_b_83127_tn_0,7728900,12,25768.0,09e16227fc42c4fe90204a9d11581034,893cb960f2ca78f823bd67adee30eaa3,25751


Good, but we don't have distance (and thus speed) available...

get from cut segments?